# Basic_Heuristic_Procedure

In [1]:
import pickle
import random
import time
import copy
import numpy as np
from collections import Counter

# Parameters is a file with data
import Parameters as params
import warnings
warnings.filterwarnings("ignore")

In [2]:
def import_data(dir_data, file_name):
    """THis function imports the data that it is in dir_data+file_name"""
    with open (dir_data+file_name, "rb") as fp:
        return pickle.load(fp)

In [3]:
def export_data(file_name, file_data):
    """Esta funcion me permite salvar en un fichero con nombbre file_data la información contenida en file_data"""
    with open(file_name, "wb") as fp:
        pickle.dump(file_data, fp)
    return

### Importing data

In [4]:
dir_data = params.dir_data

In [5]:
# Candidate locations
Nc = import_data(dir_data, "ListMECInstalled")    # candidate coordenadas
Nc = sorted(Nc , key=lambda k: [k[1], k[0]])
capacity_nc = import_data(dir_data, "Dict_C_c") 


# access nodes
Nr = import_data(dir_data, "BS_pos")
Nr = sorted(Nr , key=lambda k: [k[1], k[0]])

# Set of all nodes
N = Nc + Nr                                     


#VNFs data
capacity_ct = import_data(dir_data,'Dict_C_t')             # VNF capacity
Dict_Tproc = import_data(dir_data,'Dict_Tproc')            # VNF processing time
T_an = params.T_an                      # Processing time of access nodes (RTT)  (us)
T_dn = params.T_dn                      # Processing time of data network (us)
M_t = import_data(dir_data,'M_t')       # M_t: Max number of intances for a VNF type 
T = params.Type_sfc                     # T: Types of VNFs
alpha = params.ALPHA

#Importing paths data 
Edges = import_data(dir_data,"Edges")                              # dict with key nodes_id (en & ran) and value latency
Dict_Links_Capacity = import_data(dir_data,"Dict_Links_Capacity")  # dict with key nodes_id (en & ran) and value bandwidth                                                               # and value 
Paths = import_data(dir_data,"Paths")
Paths_nm = import_data(dir_data,"Paths_nm")                        # dict  with key nodes_id (en & ran) and value number of paths
Paths_links_mapping = import_data(dir_data,'Paths_links_mapping')  # W_p_uv: 1 if link uv belongs to path p


# Session information 
S = import_data(dir_data, "S")                    # Number of users (i.e., PDU sessions or SFCR)
users_sourceBS = import_data(dir_data, "users_sourceBS")            # source BS of each user/session
SessionInfo_h = import_data(dir_data, 'SessionInfo_S'+str(S)) 
SessionInfo_h['SourceBS'] = [Nr.index(k)+1+len(Nc) for k in users_sourceBS.values()][:S]


# Current placement conditions
Paths_sfc_vnf_h = import_data(dir_data, "Paths_sfc_vnf")
Paths_sfc_ran_h = import_data(dir_data, "Paths_sfc_ran")   
Dict_Links_availableCap_h = import_data(dir_data,'Dict_Links_availableCap') 
vnfType_location = import_data(dir_data, "vnfType_location")        # dict:c key & list of assigned tic
Xtic_h = import_data(dir_data, "Xtic")  
Xtic_h_capacity = import_data(dir_data, "Xtic_capacity")  # xtix available capacity
Xtic_h_sessions = import_data(dir_data, "Xtic_sessions")  # dict: tic key & list of assigned sessions with index >= 1
Zsfc_h = import_data(dir_data, "Zsfc")

In [6]:
flag_extrasfc_upf = params.FLAG_EXTRASFC_UPF  #allow unmapping also those sessions assigned to UPFS with low 
                                              #capacity in case of partial heuristic

##  Transforming data

In [7]:
initial_instances_id = {}
initial_instances_loc = {}

for t,i,c  in Xtic_h:
    initial_instances_id.setdefault(t, []).append(i)
    initial_instances_loc[(t,i)] = c
    
initial_VNFid_loc = {}  #For each c, it has a dict with key type and values ids.  

for c, v in vnfType_location.items():
    initial_VNFid_loc.setdefault(c, {})
    for t,i,c in v:
        initial_VNFid_loc[c].setdefault(t, []).append(i)

#### Cost components normalization and weights

In [9]:
# weight factors
w_a = params.WEIGHT_D_EN            # NODE ACTIVATION
w_r = params.WEIGHT_D_RUN           # VNF operation
w_d = params.WEIGHT_D_DEP           # VNF deployment
w_m = params.WEIGHT_D_MIG           # VNF migration
w_l = params.WEIGHT_D_ROU           # traffic routing
w_s = params.WEIGHT_D_REA           # session reassignment

WeightFactors = [w_a, w_r, w_d, w_m, w_l, w_s]

# Normalization parameters
Branches = sum(SessionInfo_h['Branches']) 
max_latency = 100*Branches          # 100 is the average Lprop budget in one way
max_upf = sum(M_t.values())         # Max number of VNFs is conditioned by M_t, which only includes upfs and not ran
max_server = len(capacity_nc)
max_newupf = max_upf-len(Xtic_h_capacity)       # MAx number of new upf that can be deployed
if max_newupf == 0:
    max_newupf = 1
max_migrations = len(Xtic_h_capacity)           # MAx number of migration depends of current deployed VNFs
max_relocations = S

NormalizationValues = [max_server, max_upf, max_newupf, max_migrations, max_latency, max_relocations]
print('NormalizationValues', NormalizationValues)

[0.167, 0.167, 0.167, 0.167, 0.166, 0.166]
NormalizationValues [13, 12, 5, 7, 3600, 25]


In [10]:
def generating_initialSolution():
    """ This function generates the initial solution which will be used during the placement reconfiguration"""
    
    #Initialing solution variables
    realcapacity_servers = copy.deepcopy(capacity_nc)
    open_servers = []
    vnf_instances = {}    # key=type of VNF and the value=list of instances id
    vnf_cap = {}
    vnfinstance_server ={} # dict of dict, main dict key=node id, value = dict key=vnf_type & value=instance id
    vnf_sfc = {}           # dict of dict, main dict key=type, valueis a dict key=instance_id & value=sfc id

    for tic in Xtic_h_capacity.keys():
        t,i,c = tic
        if c not in open_servers:
            open_servers.append(c)

        vnf_instances.setdefault(t, [])
        vnf_instances[t].append(i)

        vnf_cap.setdefault(t, {})
        vnf_cap[t][i] = round(Xtic_h_capacity[tic],2)  # Xtic_h_capacity available

        vnfinstance_server.setdefault(c, {})
        vnfinstance_server[c].setdefault(t, [])
        vnfinstance_server[c][t].append(i)

        vnf_sfc.setdefault(t, {})
        vnf_sfc[t][i] = Xtic_h_sessions[tic]

        realcapacity_servers[c] = round(realcapacity_servers[c] - capacity_ct[t], 2)


    sfc_mapping = {}
    for s,f,c  in Zsfc_h:
        t = SessionInfo_h['Type'][s-1][f-1]

        for tic, users in Xtic_h_sessions.items():
            t1,i1,c1 = tic      
            if ((t == t1 ) and (c == c1)) and s in users:
                sfc_mapping.setdefault(s, {}).setdefault('VNFs', {})
                sfc_mapping[s]['VNFs'][f] = tic

    for s in range(1, S+1):
        sfc_mapping[s]['Paths'] = [(Paths_sfc_ran_h[s])]
        if s in Paths_sfc_vnf_h.keys():
            sfc_mapping[s]['Paths'] += list(Paths_sfc_vnf_h[s].values())

    solution = {}
    solution['LinksCap'] = copy.deepcopy(Dict_Links_availableCap_h)       
    solution['ServersCap'] = realcapacity_servers
    solution['OpenServers'] = open_servers
    solution['VNFInstances'] = vnf_instances
    solution['VNFCapacity'] = vnf_cap
    solution['NodeVNFInstances'] = vnfinstance_server
    solution['VNFSFCAssigned'] = vnf_sfc
    solution['SFCMapping'] = sfc_mapping

#     # validating  sfc_mapping: commented to reduce Texec
#     for s in range(1, S+1):
#         if len(sfc_mapping[s]['VNFs']) != len(SessionInfo_h['VNFs'][s-1]):
#             print('ERROR: review SFC mapping')
#             raise
    
    return solution

#### Defining PDU sessions  parameters

In [11]:
def gathering_infoSFC(SessionInfo_h):
    """ This function checks the current values of latency for each sfc in order to determine the sessions
    that will be reassigned"""
    
    BS_popularity = dict(Counter(list(SessionInfo_h['SourceBS'])))
    
    # Gathering Info about the current sfc configuration
    Lv_list = []           # 1 if a session has poor QoS (latency violation); 0 otherwise
    Latency_budget_list = []
    Lv_users_list = []
    BS_pop_list = []
    
    # verifying latency in the user plane
    for user in range(1,S+1):
        lat = 0
        ind_lv = 0
        for b in range(SessionInfo_h['Branches'][user-1]):
            t_proc = 2*sum(p*Dict_Tproc[u] for p,u in zip(SessionInfo_h['Presence'][user-1][b], 
                                                         SessionInfo_h['Type'][user-1])) + T_dn + T_an
            t_prop = 2*(sum(SessionInfo_h['Order'][user-1][b][f-1][g-1]*Paths[Paths_sfc_vnf_h[user][f,g]] 
                            for f,g in Paths_sfc_vnf_h.get(user, {}).keys()) + Paths[Paths_sfc_ran_h[user]])
            latency = round(t_proc + t_prop,2)
            if latency > lat:
                lat = latency
        
        Latency_budget_list.append(SessionInfo_h['Latency'][user-1] - lat)

        if lat > SessionInfo_h['Latency'][user-1]:
            Lv_users_list.append(user)
            ind_lv = 1
            
        Lv_list.append(ind_lv)
        BS_pop_list.append(float(str(BS_popularity[SessionInfo_h['SourceBS'][user-1]]) + '.'+ 
                                 str(SessionInfo_h['SourceBS'][user-1])))
        

    SessionInfo_h['Lbudg'] = Latency_budget_list
    SessionInfo_h['Lv'] = Lv_list
    SessionInfo_h['BS_pop'] = BS_pop_list

    return SessionInfo_h, Lv_users_list


In [12]:
def determine_candidates(source_vnf, Lprop_vnf, Available_c):
    """This fucntion determines the candidates for the placement of a given vnf.
    Inputs:
    - source_vnf: The id of the source node (the previous vnf in the branch of the current vnf)
    - Lprop_vnf: the latency budget for the propagation delay
    - Available_c: those candidates that have capacity to instatiate a vnf or to serve a request
    Output:
    - List_c: A list with the position of possible candidates
    """      
    List_c = []
    
    for kp, vp in Paths.items():
        if kp[0] == source_vnf and vp <= min(Lprop_vnf):
            if (kp[1] not in List_c ) and (kp[1] in Available_c):
                List_c.append(kp[1]) 
                
    return List_c

In [13]:
def critic_classifier(session_info, available_c):
    """This function classified the sessions as critic or not depending on their number of available candidates
    Input:
    - session_info: DF with info about sessions
    - available_c: List of available candidates for the VNf placement
    """
    
    List_critic_s = []  # 1 if the session is in a critical state (few available candidates), 0 otherwise
    List_Numcand_s = []
    List_candidates_s = []
    
    for index, row in session_info.iterrows():
        aux_list = determine_candidates(row['SourceBS'], row['L_prop'], available_c)

        List_Numcand_s.append(len(aux_list))
        List_candidates_s.append(aux_list)
        if len(row['Type'])==1:
            if len(aux_list)<=1:
                List_critic_s.append(1)
            else:
                List_critic_s.append(0)
        else:
            if len(aux_list)<=2:
                List_critic_s.append(1)
            else:
                List_critic_s.append(0)

    session_info['Critic'] = List_critic_s 
    session_info['NumCand'] = List_Numcand_s 
    session_info['Candidates'] = List_candidates_s 
    
    return session_info

In [14]:
def sfc_releaseResources( UnmapSessionInfo_h, solution):
    """ This function is in charge of releasing resources assigned to the SFC
    """
    
    list_sfc = []
    
    for ind, row in UnmapSessionInfo_h.iterrows():
        # 1- Unmap sfc to release its assigned resources (release upfs and links capacity)
        for t,i,c in solution['SFCMapping'][row['SessionId']]['VNFs'].values():
            solution['VNFCapacity'][t][i] = round(solution['VNFCapacity'][t][i]  + row['Demand'],2)

            if round(solution['VNFCapacity'][t][i],2) == capacity_ct[t]:
                # update node_vnf_instance and vnf_instances
                if len(solution['NodeVNFInstances'][c][t]) == 1:
                    del solution['NodeVNFInstances'][c][t]
                    if sum([len(v) for v in solution['NodeVNFInstances'][c].values()]) == 0:
                        del  solution['NodeVNFInstances'][c]
                else:
                    solution['NodeVNFInstances'][c][t].remove(i)
                    
                #step: deleting instance from 'VNFInstances'
                if len(solution['VNFInstances'][t]) == 1:
                    del solution['VNFInstances'][t]
                else: 
                    solution['VNFInstances'][t].remove(i)

                if i in solution['VNFInstances_tmp'].get(t, []):
                    solution['VNFInstances_tmp'][t].remove(i)
                    if len(solution['VNFInstances_tmp'][t]) == 0:
                        del solution['VNFInstances_tmp'][t]

                # update server capacity:
                solution['ServersCap'][c] = round(solution['ServersCap'][c] + capacity_ct[t], 2)
                if solution['ServersCap'][c] == capacity_nc[c]:
                    solution['OpenServers'].remove(c)  
#                     print('empty server:', c)

        for path in solution['SFCMapping'][row['SessionId']]['Paths']:
            for link, map_ind in Paths_links_mapping[path].items():
                if map_ind == 1:  
                    # update link available capacity
                    solution['LinksCap'][link] += row['Bw']

        del solution['SFCMapping'][row['SessionId']]
        list_sfc.append(row['SessionId'])
           
    for t, v in solution['VNFSFCAssigned'].items():
        for i,users in v.items():
            solution['VNFSFCAssigned'][t][i] = list(set(users) - set(list_sfc))

    return solution

In [15]:
def unmapping_sessions(solution_y, UnmapSessionInfo_h, MappedSessionInfo_h, flag_extrasfc_upf):
    """This function unmaps the resources used for sessions in the current placement consideration.
    If all the sessions are decided to be unmapped, then this procedure can be omitted.
    It will return the set of unmapped sessionInfo updated and the solution with released resources.
    This function is different from the one used to reassign sessions. Specifically, it is called only at 
    the begining to prepare the initial solution."""
    
    solution_ = copy.deepcopy(solution_y)
    # releasing the resources of sessions selected due to latency violations
    solution_ = sfc_releaseResources(UnmapSessionInfo_h, solution_)
     
    if flag_extrasfc_upf:
        # verifying capacity of the VNF
        unmapedsfc = pd.DataFrame()
        for t, v in solution_['VNFCapacity'].items():
            for i, cap in v.items():
                if (capacity_ct[t] - cap )/ capacity_ct[t] < params.UPF_CAP_MIN:
                    for s in solution_['VNFSFCAssigned'][t][i]:
                        unmapedsfc = unmapedsfc.append(MappedSessionInfo_h[MappedSessionInfo_h['SessionId']==s])
                        MappedSessionInfo_h.drop(MappedSessionInfo_h.index[(MappedSessionInfo_h['SessionId'] == s)],
                                               axis = 0, inplace=True)

        solution_ = sfc_releaseResources(unmapedsfc, solution_)

        UnmapSessionInfo_h = UnmapSessionInfo_h.append(unmapedsfc)        

    return solution_, UnmapSessionInfo_h, MappedSessionInfo_h

## Mapping Procedure: components

In [16]:
def verify_candidate(solution_x , c, source_vnf, vnf_id, vnf_type, row, vnf_relocat_):
    """This function verifies that none constraint is violated if a vnf is placed at c location. 
    It also verifies the existence of a path with available capacity. In case of more than one path it returns 
    the shortest one.
    Inputs:
    -c: candiate node(id)
    Outputs:
    -List_const: list with number of constraints violated. If no constraint is exceeded, it is empty
    -Selectd_path: selected path for the routing. default value=none
    -VariableValues: Binary Tuple (act-ser, run-vnf, new-vnf, mig, rout, reloc). Default values  [0,0,0,0,0,0] 
    """
    List_const = []
    Selected_path = None
    VariableValues =  [0,0,0,0,0,0]   # act-ser, run-vnf, new-vnf, mig, rout, reloc
    
    # verifying the existence of a vnf instance of same type in the candidate
    Instances_c = solution_x['NodeVNFInstances'].get(c, {}).get(vnf_type, None)
    VNFCapFailure = False     
    if Instances_c != None:  #There is at least one VNf instance of the same type. Thereby, verify instances capacity
        VNFCapFailure = True
        for inst in Instances_c:
            if solution_x['VNFCapacity'][vnf_type][inst] - row['Demand'] >= (1-alpha)*capacity_ct[vnf_type]:
                VNFCapFailure = False
                break
                
    if Instances_c == None or VNFCapFailure: #There is not instances or they do not have capacity
        if solution_x['ServersCap'][c] < capacity_ct[vnf_type]:  #the candidate does not have more capacity
            List_const.append('C1')
            return List_const, Selected_path, VariableValues
            if VNFCapFailure:
                List_const.append('C2')
                return List_const, Selected_path, VariableValues
        else:
            # We should create a new instance of vnf_type, but before we need to verify the max num instances
            if (len(solution_x['VNFInstances'].get(vnf_type,[]))+1) > M_t[vnf_type]:
                List_const.append('C4')
                return List_const, Selected_path, VariableValues
    
    # verifying UPF constraints
    if vnf_type==1:  # It's a PSA UPF
        for v in solution_x['SFCMapping'].get(row['SessionId'], {}).get('VNFs', {}).values():
            # Anti-affinity constraint:
            if v[0]==vnf_type and v[2]==c:   # v--> (type, instance, c)
                List_const.append('C16')
                return List_const, Selected_path, VariableValues
            # PSA_IUPF constraint:
            elif v[0]==2 and v[2]==c:   # v--> (type, instance, c)
                List_const.append('C17')
                return List_const, Selected_path, VariableValues
                
    
    # finding the shortest path with available capacity
    pathid, pathdelay = [], []
    
    for k,v in Paths.items():
        if k[0]==source_vnf and k[1]==c:
            pathid.append(k)
            pathdelay.append(v)
            
    SortedPaths = [x for _,x in sorted(zip(pathdelay, pathid))]   # sorting the paths by their prop delay        
    
    #checking path capacity
    for path in SortedPaths:
        LinkCapFailure = False
        for link,v in Paths_links_mapping[path].items():
            if v==1 and solution_x['LinksCap'][link] < row['Bw']:
                LinkCapFailure = True
                break
        if not LinkCapFailure:
            Selected_path = path
            break
            
    if Selected_path==None: 
        List_const.append('C3')
        return List_const, Selected_path, VariableValues
    
    
    # If we are at this step, it is because len(list_const)=0: No constraint violation

    # Determing the VariableValues = [0,0,0,0,0,0]   # act-ser, run-vnf, new-vnf, mig, rout, reloc
    if solution_x['NodeVNFInstances'].get(c, None) == None:
        VariableValues[0], VariableValues[1] = 1, 1
        # Checking previous placement to reuse vnfs if any available. This avoid new deployment and migration
        instance = None
        if initial_VNFid_loc.get(c, {}).get(vnf_type, None) != None: 
            for i in initial_VNFid_loc[c][vnf_type]:
                if i not in solution_x['VNFInstances'].get(vnf_type,[]):
                    instance = i
                    break

        # otherwise, 'determine' f it is a migration or new deploy
        if instance == None:
            if len(solution_x['VNFInstances'].get(vnf_type,[])) >= len(initial_instances_id.get(vnf_type,[])):
                VariableValues[2] = 1
            else:
                # Verifying if there is available instances from previous placement,to use same id and
                # avoid migration
                VariableValues[3] = 1   # start assuming migraton
                for ind in initial_VNFid_loc.get(c, {}).get(vnf_type, []):
                    if ind not in solution_x['VNFInstances'].get(vnf_type, []):
                        VariableValues[3] = 0  # no need of migration, we can use instance with same type & id
                        break                    

    elif VNFCapFailure or Instances_c == None:
        VariableValues[1] = 1
        # Checking previous placement to reuse vnfs if any available. This avoid new deployment and migration
        instance = None
        if initial_VNFid_loc.get(c, {}).get(vnf_type, None) != None: 
            for i in initial_VNFid_loc[c][vnf_type]:
                if i not in solution_x['VNFInstances'].get(vnf_type,[]):
                    instance = i
                    break

        # otherwise, 'determine' f it is a migration or new deploy
        if instance == None:
            if len(solution_x['VNFInstances'].get(vnf_type,[])) >= len(initial_instances_id.get(vnf_type,[])):
                VariableValues[2] = 1
            else:
                # Verifying if there is available instances from previous placement,to use same id and
                # avoid mig
                VariableValues[3] = 1   # start assuming migraton
                for ind in initial_VNFid_loc.get(c, {}).get(vnf_type, []):
                    if ind not in solution_x['VNFInstances'].get(vnf_type, []):
                        VariableValues[3] = 0  # no need of migration, we can use instance with same type & id
                        break 

    VariableValues[4] = Paths[Selected_path] 

    for sfc in Zsfc_h:
        if sfc[0] == row['SessionId'] and sfc[1] == vnf_id and c != sfc[2]:
            VariableValues[5] = 1
            break
    
    # The following steps were omitted to save time
#     if VariableValues[2] == 1 and VariableValues[3] == 1:
#         print('ERROR, there is either a new deployment or migration, but no both at the same time')
#         raise
        
    return List_const, Selected_path, VariableValues

In [17]:
def simulating_vnfdeployment(solution_2, c, vnf_id, vnf_type, Selected_path, VariableValues, row):
    """This function will simulate the deployment of vnf in c to analize the deployment posibilities for 
    the next vnf if the current location is selected.
    1-To simulate the deployment we will create a copy of the sets and add the mapping of vnf to links and nodes
    2-Find next VNF in the branch 
    3-Take c as source for vnf+1 and determine the candidates
    3-Evaluate each candidate
    4-Select the best candidate
    """  
    #Updating the Sets
    if VariableValues[0]==1: # Opening of a new server   
        solution_2['OpenServers'].append(c)
        
    if VariableValues[1]==1: # create a new instance of vnf_type  (new deploymnet or migrtion)
        solution_2['ServersCap'][c] -= capacity_ct[vnf_type]
        
        # determine index of instance (new or mig)
        if VariableValues[2] == 1 or VariableValues[3] == 1:   # new deployment or migration
            for ind in range(len(solution_2['VNFInstances_tmp'].get(vnf_type, []))+1):
                if  'tmp' + str(ind+1) not in solution_2['VNFInstances_tmp'].get(vnf_type, []):
                    instance = 'tmp' + str(ind+1)
                    break
#             print('temporal id', instance)
#             print(solution_2['VNFInstances_tmp'].get(vnf_type, []))
            solution_2['VNFInstances_tmp'].setdefault(vnf_type,[]).append(instance) 
            
        else: 
            for ind in initial_VNFid_loc.get(c, {}).get(vnf_type, []):
                if ind not in solution_2['VNFInstances'].get(vnf_type, []):
                    instance = ind
                    break 
           
        solution_2['VNFInstances'].setdefault(vnf_type,[]).append(instance)
        solution_2['VNFCapacity'].setdefault(vnf_type,{})[instance] = capacity_ct[vnf_type]
        solution_2['NodeVNFInstances'].setdefault(c, {}).setdefault(vnf_type,[]).append(instance)
    
    else:  # find instance with capacity
        for instance in solution_2['NodeVNFInstances'][c][vnf_type]:
            if solution_2['VNFCapacity'][vnf_type][instance] - row['Demand'] >= (1-alpha)*capacity_ct[vnf_type]:
                break
    
    # update capacity
    solution_2['VNFCapacity'][vnf_type][instance] = round(solution_2['VNFCapacity'][vnf_type][instance]
                                                           - row['Demand'],2)  
    solution_2['SFCMapping'].setdefault(int(row['SessionId']), {}).setdefault('VNFs', {})[vnf_id] = (vnf_type, 
                                                                                                instance, c)
    solution_2['SFCMapping'].setdefault(int(row['SessionId']), {}).setdefault('Paths', []).append(Selected_path)
    solution_2['VNFSFCAssigned'].setdefault(vnf_type,{}).setdefault(instance,[]).append(int(row['SessionId']))

    for link,v in Paths_links_mapping[Selected_path].items():
        if v==1: #update link capacity
            solution_2['LinksCap'][link]-= row['Bw']
    
    return solution_2

In [22]:
def evaluate_candidates(solution_x, candidates_set, source, v_id, v_type, row, vnf_relocat_):
    """Determines which candidates in the set are feasible and which one not for the deployment of 
    the selected vnf
    (act-ser, run-vnf, new-vnf, mig, rout, reloc)
    """

    feasible_cadidates = {}  

    #Verify candidate
    for c in candidates_set:
        list_const, selected_path, variablevalues = verify_candidate(solution_x, c, source,  v_id,  v_type, row,
                                                                     vnf_relocat_)  
        
        if len(list_const) == 0:   #Feasible candidate, no constraint was violated
            cost_c = sum([(a*b)/c for a,b,c in zip(WeightFactors, variablevalues, NormalizationValues)]) 

            feasible_cadidates[c] = [selected_path, variablevalues, cost_c]

    return feasible_cadidates

In [27]:
def mapping_procedure(solution_copy, row, available_c, AVAILABLE_NC):
    """ This function is in charge of mapping the vnfs in the sfcr, it will return a flag indicating the 
    state (failure or success) of the mapping procedure and a dict containing the data  associated  to the
    output status"""
        
    flag_mapping_error = False      # pdt
    VNFLoc = {}                     # dict vnf_id as key and c, lprop as value
    vnf_relocat = {}                # dict vnf_id as key and binary indicator as value(0 no relocated)

    for f_id, f_type  in zip(row['VNFs'], row['Type']):            # f_id , f_typev= vnf_id, vnf_type
        if f_id not in VNFLoc.keys():  # If the VNF has not been mapped yet, analyze its placement solut.
            best_cost = 1000           # set to a high value
            destination_vnf_f = []
            
            for branch, Lprop in zip(row['Presence'], row['L_prop']):
                # Steps 9-11: Select the candidates that are common to all the source according to Lprop.
                # In our case, the source is the same for all vnf. Thus, we just need to update the candidates.
                if f_id == 1:
                    source_vnf_f = row['SourceBS'] # The BS is the initial source for all the branches
                    Lprop_vnf_f = [Lprop]          # This is also updated with the placement of each VNf in a branch
                    candidates_f = row['Candidates'] #For the first vnf they satisfy latency req
                else:
                    source_vnf_f = VNFLoc[1][0]    # In the case under study, it's the same source (first f)
                    Lprop_vnf_f = VNFLoc[1][1]     # In the case under study,it's the same source (first f)
                    candidates_f = determine_candidates(source_vnf_f, Lprop_vnf_f, available_c)

                # determining if f is the last: In our specific case, is easier to see the type of the vnf. 
                # If it's the last vnf, we do not need to determine the destination_vnfs. PSA  (f_type =1) is the last vnf
                #  in the chain
                if f_type != 1:                             
                    for next_f_index in range(f_id,len(row['VNFs'])):  # f_ids tarts in 1, thus if we use it to 
                                                                       # iterate,we will be looking at the next vnfs.
                        if branch[next_f_index] == 1:     
                            destination_vnf_f.append(row['VNFs'][next_f_index])
                            break  
              
            # steps 12-14: Evaluate candidates
            feasible_cadidates = evaluate_candidates(solution_copy,candidates_f, source_vnf_f, f_id,
                                                     f_type, row, vnf_relocat) 

            # steps 17: sort feasible candidates by their cost in ascending order
            sorted_candidates_f = sorted(feasible_cadidates.keys(), key=lambda k: feasible_cadidates[k][2])
            
            for c in sorted_candidates_f:
                selected_path, variablevalues, cost = feasible_cadidates[c]
                if cost < best_cost:
                    # step 20 & 27: simulating deployment of vnf f
                    VNFLoc_aux = copy.deepcopy(VNFLoc)  
                    vnf_relocat_aux = copy.deepcopy(vnf_relocat)
                    solution_copy2 = copy.deepcopy(solution_copy)
                    solution_copy2 = simulating_vnfdeployment(solution_copy2, c, f_id, f_type,
                                                              selected_path, variablevalues, row)
                    #steps 15-: Looking-ahead
                    if f_type !=1:                                
                        Lprop_next_f = [Lprop_vnf_f[0] - Paths[selected_path]] 
                        candidates_next_f = determine_candidates(c, Lprop_next_f, available_c)
                        VNFLoc_aux[f_id] = (c, Lprop_next_f) 
                        vnf_relocat_aux[f_id] = variablevalues[-1]
                        
                        for next_f_id in destination_vnf_f:
                            # steps 24-25: Evaluate candidates
                            feasible_cadidates_next_f = evaluate_candidates(solution_copy2, 
                                                                            candidates_next_f, c, 
                                                                            next_f_id, 
                                                                            row['Type'][next_f_id-1], row, 
                                                                            vnf_relocat_aux)
                            
                            if len(feasible_cadidates_next_f) != 0:
                                # steps 26: selecting the best candidates for the next vnf                            
                                best_c_next_f = sorted(feasible_cadidates_next_f.keys(), key=lambda k: 
                                                             feasible_cadidates_next_f[k][2])[0]
                                
                                # Step 27: simulating deployment of next vnf
                                solution_copy2 = simulating_vnfdeployment(solution_copy2, best_c_next_f, 
                                                                          next_f_id, row['Type'][next_f_id-1],
                                                                          feasible_cadidates_next_f[best_c_next_f][0], 
                                                                          feasible_cadidates_next_f[best_c_next_f][1],
                                                                          row)
                                # Step 28: Update candidate Cost
                                cost += feasible_cadidates_next_f[best_c_next_f][2]
                                VNFLoc_aux[next_f_id] = (best_c_next_f, Lprop_next_f[0]-
                                                         Paths[feasible_cadidates_next_f[best_c_next_f][0]]) 
                                vnf_relocat_aux[next_f_id] = feasible_cadidates_next_f[best_c_next_f][1][-1]
                                
                            else: 
                                cost += 1000    
                                break

                        if cost < best_cost:
                            best_cost = cost
                            best_solution = copy.deepcopy(solution_copy2)
                            VNFLoc_best = copy.deepcopy(VNFLoc_aux) 
                            vnf_relocat_best = copy.deepcopy(vnf_relocat_aux)
                    else:
                        best_cost = feasible_cadidates[c][2] 
                        best_solution = copy.deepcopy(solution_copy2)
                        VNFLoc_best = copy.deepcopy(VNFLoc_aux) 
                        VNFLoc_best[f_id] = (c, [Lprop_vnf_f[0] - Paths[feasible_cadidates[c][0]]])  #pdt
                        vnf_relocat_best = copy.deepcopy(vnf_relocat_aux)                        
                        vnf_relocat_best[f_id] = variablevalues[-1]
                        break  
                                     
            # steps 33-34:        
            if best_cost < 1000:
                solution_copy = copy.deepcopy(best_solution)
                VNFLoc = copy.deepcopy(VNFLoc_best)
                vnf_relocat = copy.deepcopy(vnf_relocat_best)
            else:
                # Pending to implement reassign procedure(): No required for the moment
                print('error mapping-------------------')
                flag_mapping_error = True
                print('unfeasible_candidates', row['SessionId'], unfeasible_candidates)
                print('feasible_cadidates', row['SessionId'], feasible_cadidates)
                print('REASSIGN PROCEDURE: vnf_type needs to be defined')
                raise
#                 flag_resources_released, solution_copy = reassign_procedure(source_vnf_f, Lprop_vnf_f, vnf_type, 
#                                                                             flag_look_ahead, solution_copy, row,
#                                                                             AVAILABLE_NC,)
                break
                        
    return flag_mapping_error, solution_copy

### Main Procedure

In [28]:
def DPC_UPCR(MappedSessionInfo_h, UnmapSessionInfo_h, Sort_criteria_list, Sort_order_list, solution_, 
                available_nc):

    #Initial variables
    SFCUnmapped = []
    Count_reassig = 0
    AVAILABLE_NC = list(capacity_nc.keys())

    # updating available candidates for each user upon their new BS and determing criticism
    MappedSessionInfo_h = critic_classifier(MappedSessionInfo_h, available_nc)
    UnmapSessionInfo_h = critic_classifier(UnmapSessionInfo_h, available_nc)  
    
    #Sorting SFCR
    if len(Sort_criteria_list) != 0:
        UnmapSessionInfo_h = UnmapSessionInfo_h.sort_values(by=Sort_criteria_list, ascending=Sort_order_list)
    
    while len(UnmapSessionInfo_h) != 0:
        solution_copy = copy.deepcopy(solution_)
        row = UnmapSessionInfo_h.iloc[0]           # selecting the first SFCR in the sorted  set

        # Procedure: mapping
        flag_mapping_error, solution_copy = mapping_procedure(solution_copy, row, available_nc, AVAILABLE_NC)
        # Procedure: reassign
        if flag_mapping_error:
            print('ERROR: reassign due to unfeasible placement: not implemented yet')
            raise
            flag_reasing = False
            Count_reassig += 1

        # updating the set of SFCRs
        UnmapSessionInfo_h = UnmapSessionInfo_h.drop(UnmapSessionInfo_h.index[0]) 
        
        # steps 40- : Update resources and (un)mapping set            
        if not flag_mapping_error:  
            solution_ = copy.deepcopy(solution_copy)
            MappedSessionInfo_h = MappedSessionInfo_h.append(row)
            #checking if we need o delete some candidate
            flag_sort_sfcr = False
            for c, instances in solution_['NodeVNFInstances'].items():
                if solution_['ServersCap'][c] == 0 and c in available_nc:   # Analize available capacity of VNFs in c
                    flag_update_c = True
                    for typ,id_f in instances.items():
                        for f in id_f:
                            if solution_['VNFCapacity'][typ][f] > (1-alpha)*capacity_ct[typ]:
                                flag_update_c = False
                                break
                        if not flag_update_c: break
                    if flag_update_c:
                        flag_sort_sfcr = True
                        available_nc.remove(c) 
                        
            if flag_sort_sfcr and len(Sort_criteria_list) != 0:
                UnmapSessionInfo_h = critic_classifier(UnmapSessionInfo_h, available_nc)                
                #Sorting SFCR 
                UnmapSessionInfo_h = UnmapSessionInfo_h.sort_values(by=Sort_criteria_list,
                                                                    ascending=Sort_order_list)        
        else:
            SFCUnmapped.append(row['SessionId'])              
            raise
    
    return solution_, SFCUnmapped, MappedSessionInfo_h, Count_reassig

### Release Assigned resources

This step is executed for the first placement, only for the reconfiguration without improvement

In [29]:
def preparing_initialSolution(SessionInfo_h):
    """ This function creates the initial solution according to the value of FLAG_ALL_SFC.
    If True, an empty solution (all thecapacity is available) is created, otherwise, a protion of sfc are 
    selected and their assigned resources are released"""

    if params.FLAG_ALL_SFC:  #if true all the sessions are analyzed during the reconfiguration. 
                             #Otherwise, the number of sessions depends of LV and ROUT_SESS
        solution_ = {}
        solution_['ServersCap'] = copy.deepcopy(capacity_nc)
        solution_['LinksCap'] = copy.deepcopy(Dict_Links_Capacity)
        solution_['OpenServers'] = []
        solution_['VNFInstances'] = {} 
        solution_['VNFCapacity'] = {}   
        solution_['NodeVNFInstances'] = {} 
        solution_['VNFSFCAssigned'] = {} 
        solution_['SFCMapping'] = {}
        solution_['VNFInstances_tmp'] = {}  #this dic is used during the evauluation process to avoid 
                                            # misleading information, consider migrations
            
        SessionInfo_h, Lv_users_list = gathering_infoSFC(SessionInfo_h) # active if sort sessions per Lbudg
    
        UnmapSessionInfo_h = copy.deepcopy(SessionInfo_h)
        MappedSessionInfo_h = pd.DataFrame(columns=list(SessionInfo_h.columns))

        
    else: 
        solution_ = generating_initialSolution()
 
        solution_['VNFInstances_tmp'] = {}  #this dic is used during the evauluation process to avoid 
                                            # misleading information, consider migrations
        
        # Unmapping sessions with Latency criteria
        SessionInfo_h, Lv_users_list = gathering_infoSFC(SessionInfo_h)       
        SortedSessionInfo_h = SessionInfo_h.sort_values(by=['Lbudg'], ascending=[True])    #Sorting SFCR 

        unmap_sess_length = min(len(Lv_users_list) + int(params.LV_EXTRA_SESS*S), S)
        UnmapSessionInfo_h = SortedSessionInfo_h[:unmap_sess_length]      # sessions to be mapped
        MappedSessionInfo_h = SortedSessionInfo_h[unmap_sess_length:]     # sessions to keep mapped

        solution_, UnmapSessionInfo_h, MappedSessionInfo_h = unmapping_sessions(solution_, 
                                                                                UnmapSessionInfo_h,
                                                                                MappedSessionInfo_h,
                                                                                flag_extrasfc_upf)

#     if len(UnmapSessionInfo_h) + len(MappedSessionInfo_h) != S:
#         print('ERROR: Verify mapped and unmapped sessions')
#         raise

    # Determining candidates with available capacity. This also includes the ones with available capacity
    # in the VNFS    
    available_nc = [ x for x in capacity_nc.keys() if x not in solution_['OpenServers']]

    for c, v in solution_['NodeVNFInstances'].items():
        for t, v1 in v.items():
            for i in v1:
                if solution_['VNFCapacity'][t][i] > (1-alpha)*capacity_ct[t] and c not in available_nc:
                    available_nc.append(c)
                    break

#     print('no.unmapped sfc', len(UnmapSessionInfo_h))
    
    return  solution_, UnmapSessionInfo_h, MappedSessionInfo_h, available_nc

## Solution

In [30]:
#Sorting SFCR
sort_criterion = params.SORT_CRITERION  # Latency

#Approach folder:latency/test4
Sort_criteria_list = ['Critic', sort_criterion,  'NumVNFs', 'SourceBS', 'NumCand',]
Sort_order_list = [False, True,  False, True, True]

#folder:latency/test2
#     Sort_criteria_list = ['Critic', sort_criterion,  'NumVNFs', 'BS_pop', 'NumCand',]
#     Sort_order_list = [False, True,  False, False, True] 

start_time = time.process_time()

solution_, UnmapSessionInfo_h, MappedSessionInfo_h, available_nc = preparing_initialSolution(SessionInfo_h)

best_solution_, SFCUnmapped, MappedSessionInfo_h, Count_reassig = DPC_UPCR(MappedSessionInfo_h, UnmapSessionInfo_h,
                                                                           Sort_criteria_list, Sort_order_list, 
                                                                           solution_, available_nc)
T_exec = time.process_time() - start_time

# Checking solution
if len(MappedSessionInfo_h) + len(SFCUnmapped) != S:
    print('ERROR: some sessions are missing')
    raise

session 4
VNFLoc_aux {1: (1, [67.5])}
VNFLoc_aux {1: (7, [11.119999999999997])}
VNFLoc_aux {1: (2, [47.19])}
VNFLoc_aux {1: (5, [46.05])}
VNFLoc_aux {1: (6, [30.4])}
VNFLoc_aux {1: (3, [29.689999999999998])}
VNFLoc_aux {1: (10, [27.180000000000007])}
VNFLoc_aux {1: (4, [26.009999999999998])}
VNFLoc_aux {1: (13, [9.680000000000007])}
VNFLoc_aux {1: (9, [6.290000000000006])}
VNFLoc_aux {1: (11, [6.0])}
session 25
VNFLoc_aux {1: (1, [26.009999999999998])}
session 5
VNFLoc_aux {1: (1, [31.0])}
session 19
VNFLoc_aux {1: (1, [10.079999999999998])}
VNFLoc_aux {1: (7, [66.46000000000001])}
session 21
VNFLoc_aux {1: (7, [47.73])}
VNFLoc_aux {1: (1, [6.320000000000007])}
session 12
VNFLoc_aux {1: (7, [31.599999999999994])}
session 24
VNFLoc_aux {1: (7, [30.39])}
session 18
VNFLoc_aux {1: (7, [31.43])}
session 22
VNFLoc_aux {1: (7, [31.43])}
session 2
VNFLoc_aux {1: (7, [27.11])}
session 14
session 17
VNFLoc_aux {1: (7, [30.840000000000003])}
session 16
VNFLoc_aux {1: (7, [59.3])}
session 7
VNFLo

## Processing the output results

In [43]:
def processing_solution(solution_, flag_print):
    
    ## Determining migrations or new deployments for the temporal VNF IDs
    unused_vnf_is = {}      # dict with the ids of unused vnfs ids per type (key) from previous placement
    for t,v in initial_instances_id.items():
        for i in v:
            if i not in solution_['VNFInstances'].get(t,[]):
                unused_vnf_is.setdefault(t, []).append(i)

    update_tmp = {}
    Num_newdeploy = 0
    Num_mig = 0
    # print('PDT to define a formula to determine the relantion between both elements')
    for t, IDs in solution_['VNFInstances_tmp'].items():
        for id_tmp in IDs: 
            if len(unused_vnf_is.get(t, [])) != 0:
                i = unused_vnf_is[t].pop(0)  # getting the first available id
                Num_mig += 1
#                 solution_['VNFInstances'][t] = [x for x in solution_['VNFInstances'][t] if x !=id_tmp]
#                 solution_['VNFInstances'][t].append(i)
            else:
                for i in range(1, M_t[t]+1):
                    if i not in solution_['VNFInstances'][t]:
                        Num_newdeploy += 1
#                         solution_['VNFInstances'][t] = [x for x in solution_['VNFInstances'][t] if x !=id_tmp]
#                         solution_['VNFInstances'][t].append(i)
                        break

            update_tmp.setdefault(t, []).append((id_tmp, i))
#     print('update_tmp, migrations, new_deploy', update_tmp, Num_mig, Num_newdeploy)

    ## Processing the  solution
    Num_upf_type = {}
    for k,v in solution_['VNFInstances'].items():
        Num_upf_type[k] = len(v)

    Total_upf = sum([len(v) for v in solution_['VNFInstances'].values()])    

    # Number of open servers
    Num_open_c = len(solution_['OpenServers'])
    Open_servers_list = solution_['OpenServers']

    #Number of Unassigned sessions 
    Unassigned= len(SFCUnmapped)
    Unassigned_s_list = SFCUnmapped 

    sfc_reloc = []
    for (s,f,c) in Zsfc_h:
        if solution_['SFCMapping'][s]['VNFs'][f][2] != c and s not in sfc_reloc:
            sfc_reloc.append(s)
    Num_reloc = len(sfc_reloc)

    latency_prop = 0
    for k,v in solution_['SFCMapping'].items():
        for branch in SessionInfo_h[SessionInfo_h['SessionId']==k]['Presence'].values[0]:
            latency_branch = sum([Paths[a]*b for a,b in zip(v['Paths'],branch)])  # one way and linearizando
            latency_prop += latency_branch

    variable_values = [Num_open_c, Total_upf, Num_newdeploy, Num_mig, latency_prop, 
                       Num_reloc]

    cost = sum([a*b/c for a,b,c in zip(WeightFactors, variable_values, NormalizationValues)])
                    
    if flag_print:
        print('   SFCs', S)
        print('   Execution time', T_exec)
        print('   Placement Cost', cost)
        print('   Number of UPF installed: ', Total_upf )
        print('   Number of new deployments', Num_newdeploy )
        print('   Number of UPFs per type', Num_upf_type )
        print('   Number of UPFs migrations', Num_mig )
        print('   Open servers list', Open_servers_list )
        print('   Number of Open Servers', Num_open_c )
        print('   Num_unassigned sessions', Unassigned)
        print('   Number of Relocations', Num_reloc )

    return (cost, Total_upf, Num_newdeploy, Num_upf_type, Num_mig, Open_servers_list, Num_open_c,
            sfc_reloc, Unassigned, Num_reloc)

In [44]:
# the initial solution is also the best, at least at the begining
(best_cost, Total_upf, Num_newdeploy, Num_upf_type, Num_mig, Open_servers_list, Num_open_c, 
 sfc_reloc, Unassigned, Num_reloc) = processing_solution(best_solution_, flag_print=True)

   SFCs 75
   Execution time 1.671875
   Placement Cost 0.24517332964812966
   Number of UPF installed:  9
   Number of new deployments 0
   Number of UPFs per type {3: 3, 1: 3, 2: 3}
   Number of UPFs migrations 0
   Open servers list [5, 9, 7]
   Number of Open Servers 3
   Num_unassigned sessions 0
   Number of Relocations 5


# Improvement phase

In [49]:
if params.FLAG_HDSFC_IMPROVE:
    SORTING_CRITERIA = [
                        ['Critic', sort_criterion,  'NumVNFs', 'SourceBS', 'NumCand',],
                        ['Critic', sort_criterion,  'NumVNFs', 'NumCand','SourceBS', ],
#                         ['Critic', sort_criterion,  'NumVNFs', ],
                        [sort_criterion],
#                         [], 
                       ]
    SORTING_ORDER = [
                     [False, True,  False, True, True],
                     [False, True,  False, True, True],
#                      [False, True,  False],
                     [True,],
#                      [], 
                    ]
    SORTING_OPTIONS = ['lat4',
                       'lat0', 
#                        'clf', 
                       'l',
#                        'unsorted'
                      ]

    flag_extrasfc_upf = params.FLAG_EXTRASFC_UPF_IH  # to avoid to empty another UPFs and get same result that H
    max_try = params.MAX_HDSFC_IMPROVE   # Max. total number of instances to check in the improvement phase  

    I_t, i, I_t_, i_ = 0,0,0,0
    
    while max_try != 0:
        max_try -= 1
        solution__ = copy.deepcopy(best_solution_)
        # option_1:
        while I_t_== I_t  and i_ == i:
            I_t = random.choice(list(solution__['VNFInstances'].keys()))
            i = random.choice(list(solution__['VNFInstances'][I_t]))        
        I_t_, i_ = I_t, i
        
        sessions_to_unmap = solution__['VNFSFCAssigned'][I_t][i]
        UnmapSessionInfo_h = pd.DataFrame()
        MappedSessionInfo_h = pd.DataFrame()
        for index, row in SessionInfo_h.iterrows():
            if row['SessionId'] in sessions_to_unmap:
                UnmapSessionInfo_h = UnmapSessionInfo_h.append(row)
            else:
                MappedSessionInfo_h = MappedSessionInfo_h.append(row)

#             if len(UnmapSessionInfo_h) != len(sessions_to_unmap):
#                 print('error')
#                 raise
#         print(len(UnmapSessionInfo_h))

        start_time = time.process_time()
# 
        tmp_solution_,UnmapSessionInfo_h,MappedSessionInfo_h = unmapping_sessions(solution__, 
                                                                                UnmapSessionInfo_h,
                                                                                MappedSessionInfo_h, 
                                                                                   flag_extrasfc_upf) 
#             if len(MappedSessionInfo_h)+len(UnmapSessionInfo_h) != S:
#                 raise

        # Update available candidates
        available_nc = [ x for x in capacity_nc.keys() if x not in tmp_solution_['OpenServers']]
        for c, v in tmp_solution_['NodeVNFInstances'].items():
            for t, v1 in v.items():
                for ix in v1:
                    if tmp_solution_['VNFCapacity'][t][ix] > (1-alpha)*capacity_ct[t] and c not in available_nc:
                        available_nc.append(c)
                        break
        # DEFINING THE SORTING CRITERIA (RANDOMLY)
        option = random.choice(range(len(SORTING_OPTIONS)))
        Sort_criteria_list = SORTING_CRITERIA[option]
        Sort_order_list = SORTING_ORDER[option]
#         print(option, SORTING_OPTIONS[option])
#         print(len(UnmapSessionInfo_h))

        tmp_solution_, SFCUnmapped, MappedSessionInfo_h, Count_reassig = DPC_UPCR(MappedSessionInfo_h, 
                                                                                  UnmapSessionInfo_h, 
                                                                                  Sort_criteria_list, 
                                                                                  Sort_order_list,
                                                                               tmp_solution_, available_nc)
#             if len(MappedSessionInfo_h)+len(SFCUnmapped) != S:
#                 raise
        if len(tmp_solution_['SFCMapping']) == S:
            (cost, Total_upf, Num_newdeploy, Num_upf_type, 
             Num_mig, Open_servers_list, Num_open_c,
             sfc_reloc, Unassigned, Num_reloc) = processing_solution(tmp_solution_, flag_print=False)
#             print('cost', cost)
            if round(cost,4) < round(best_cost,4):
#                 print('--> this solution is better', cost, best_cost)
                best_cost = cost
                best_solution_ = copy.deepcopy(tmp_solution_)
        else:
            print('unfeasible or error ', SFCUnmapped)
            raise

        T_exec += (time.process_time() - start_time)
    
    (best_cost ,Total_upf, Num_newdeploy, Num_upf_type, Num_mig, Open_servers_list, Num_open_c,
            sfc_reloc, Unassigned, Num_reloc) = processing_solution(best_solution_, flag_print=True)
    print('--------------------')
    

I_t, i 2 1
I_t_, i_ 2 1
2 l
cost 0.23560854378954377
I_t, i 1 2
I_t_, i_ 1 2
2 l
cost 0.24637942863802864
I_t, i 1 1
I_t_, i_ 1 1
1 lat0
cost 0.23782187712287714
I_t, i 2 1
I_t_, i_ 2 1
1 lat0
cost 0.23560854378954377
I_t, i 1 1
I_t_, i_ 1 1
0 lat4
cost 0.23782187712287714
I_t, i 1 2
I_t_, i_ 1 2
0 lat4
cost 0.23560854378954382
I_t, i 1 1
I_t_, i_ 1 1
2 l
cost 0.24679392560772562
I_t, i 1 3
I_t_, i_ 1 3
2 l
cost 0.23527386096126096
--> this solution is better 0.23527386096126096 0.2356085437895438
I_t, i 1 1
I_t_, i_ 1 1
1 lat0
cost 0.2425805437895438
I_t, i 1 3
I_t_, i_ 1 3
0 lat4
cost 0.23527386096126096
   SFCs 75
   Execution time 15.953125
   Placement Cost 0.23527386096126096
   Number of UPF installed:  9
   Number of new deployments 0
   Number of UPFs per type {1: 3, 2: 3, 3: 3}
   Number of UPFs migrations 0
   Open servers list [5, 9, 7]
   Number of Open Servers 3
   Num_unassigned sessions 0
   Number of Relocations 1
--------------------


### Determining migrations or new deployments

In [ ]:
## Determining migrations or new deployments
unused_vnf_is = {}      # dict with the ids of unused vnfs ids per type (key) from previous placement
for t,v in initial_instances_id.items():
    for i in v:
        if i not in best_solution_['VNFInstances'].get(t,[]):
            unused_vnf_is.setdefault(t, []).append(i)
print('unused_vnf_is', unused_vnf_is)

update_tmp = {}
Num_newdeploy = 0
Num_mig = 0
# print('PDT to define a formula to determine the relantion between both elements')
for t, IDs in best_solution_['VNFInstances_tmp'].items():
    for id_tmp in IDs: 
        if len(unused_vnf_is.get(t, [])) != 0:
            i = unused_vnf_is[t].pop(0)  # getting the first available id
            Num_mig += 1
            best_solution_['VNFInstances'][t] = [x for x in best_solution_['VNFInstances'][t] if x !=id_tmp]
            best_solution_['VNFInstances'][t].append(i)
        else:
            for i in range(1, M_t[t]+1):
                if i not in best_solution_['VNFInstances'][t]:
                    Num_newdeploy += 1
                    best_solution_['VNFInstances'][t] = [x for x in best_solution_['VNFInstances'][t] if x !=id_tmp]
                    best_solution_['VNFInstances'][t].append(i)
                    break

        update_tmp.setdefault(t, []).append((id_tmp, i))
print('update_tmp, migrations, new_deploy', update_tmp, Num_mig, Num_newdeploy)

# Updating temporary Ids due to migrations
for t, Ids_pairs in update_tmp.items():
    for id_pair in Ids_pairs:
        best_solution_['VNFCapacity'][t][id_pair[1]] = best_solution_['VNFCapacity'][t][id_pair[0]]
        del best_solution_['VNFCapacity'][t][id_pair[0]]
        best_solution_['VNFSFCAssigned'][t][id_pair[1]] = best_solution_['VNFSFCAssigned'][t][id_pair[0]]
        del best_solution_['VNFSFCAssigned'][t][id_pair[0]]
        for c, v in best_solution_['NodeVNFInstances'].items():
            if id_pair[0] in best_solution_['NodeVNFInstances'][c].get(t, []):
                best_solution_['NodeVNFInstances'][c][t] = [i for i in best_solution_['NodeVNFInstances'][c][t]
                                                       if i !=id_pair[0]]
                best_solution_['NodeVNFInstances'][c][t].append(id_pair[1])
        for s, v  in best_solution_['SFCMapping'].items():
            for f, tic in v['VNFs'].items():
                if tic[0]==t and tic[1]==id_pair[0]:
                    best_solution_['SFCMapping'][s]['VNFs'][f] = (t, id_pair[1], tic[2])